In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

tqdm.pandas()

In [ ]:
def drop_multi_index_columns(df):
    if isinstance(df.columns, pd.core.indexes.multi.MultiIndex):
        print("Drop multiindex header", type(df.columns))
        df.columns = df.columns.droplevel(0)
  

In [ ]:
# rename Datetime column to Date
def rename_datetime_to_Date(df):
    df.index.names = ['Date']
    

In [ ]:
def strategy_01(df, current_candle):
    current_pos = df.index.get_loc(current_candle)

    c1 = df['Low'].iloc[current_pos-4] > df['High'].iloc[current_pos]
    c2 = df['High'].iloc[current_pos] > df['Low'].iloc[current_pos-3]
    c3 = df['Low'].iloc[current_pos-3] > df['Low'].iloc[current_pos-2]
    c4 = df['Low'].iloc[current_pos-2] > df['Low'].iloc[current_pos-1]
    c5 = df['Close'].iloc[current_pos] > df['High'].iloc[current_pos-1]

    if c1 and c2 and c3 and c4 and c5:
        return 2

    c1 = df['High'].iloc[current_pos-4] < df['Low'].iloc[current_pos]
    c2 = df['Low'].iloc[current_pos] < df['High'].iloc[current_pos-3]
    c3 = df['High'].iloc[current_pos-3] < df['High'].iloc[current_pos-2]
    c4 = df['High'].iloc[current_pos-2] < df['High'].iloc[current_pos-1]
    c5 = df['Close'].iloc[current_pos] < df['Low'].iloc[current_pos-1]

    if c1 and c2 and c3 and c4 and c5:
        return 1  # Return 1 as the function result for these symmetrical conditions

    return 0

def add_signal(df):
    df['Signal'] = df.progress_apply(lambda row: strategy_01(df, row.name), axis=1)
    return df


In [5]:
def add_pointpos_column(df, signal_column):
    """
    Adds a 'pointpos' column to the DataFrame to indicate the position of support and resistance points.

    Parameters:
    df (DataFrame): DataFrame containing the stock data with the specified SR (support/resistance) column, 'Low', and 'High' columns.
    sr_column (str): The name of the column to consider for the SR (support/resistance) points.

    Returns:
    DataFrame: The original DataFrame with an additional 'pointpos' column.
    """
    def pointpos(row):
        if row[signal_column] == 2:
            return row['Low'] - 1e-4
        elif row[signal_column] == 1:
            return row['High'] + 1e-4
        else:
            return np.nan

    df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
    return df

In [ ]:
def add_pointpos_column(df, signal_column, coef):
    print("Params: {}".format(coef))
    def pointpos(row):
        if row[signal_column] == 2:
            return row['Low'] - coef
        elif row[signal_column] == 1:
            return row['High'] + coef
        else:
            return np.nan

    df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)
    return df

In [6]:
def plot_candlestick_with_signals(df, start_index, num_rows):
    """
    Plots a candlestick chart with signal points.

    Parameters:
    df (DataFrame): DataFrame containing the stock data with 'Open', 'High', 'Low', 'Close', and 'pointpos' columns.
    start_index (int): The starting index for the subset of data to plot.
    num_rows (int): The number of rows of data to plot.

    Returns:
    None
    """
    df_subset = df[start_index:start_index + num_rows]

    fig = make_subplots(rows=1, cols=1)

    fig.add_trace(go.Candlestick(x=df_subset.index,
                                 open=df_subset['Open'],
                                 high=df_subset['High'],
                                 low=df_subset['Low'],
                                 close=df_subset['Close'],
                                 name='Candlesticks'),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=df_subset.index, y=df_subset['pointpos'], mode="markers",
                             marker=dict(size=10, color="MediumPurple", symbol='circle'),
                             name="Entry Points"),
                  row=1, col=1)

    fig.update_layout(
        width=1200,
        height=800,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        xaxis=dict(showgrid=False, zeroline=False),
        yaxis=dict(showgrid=False, zeroline=False),
        showlegend=True,
        legend=dict(
            x=0.01,
            y=0.99,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=12,
                color="white"
            ),
            bgcolor="black",
            bordercolor="gray",
            borderwidth=2
        )
    )

    fig.show()

In [ ]:
def add_bb_rsi_signal(df, rsi_threshold_low=30, rsi_threshold_high=70, bb_width_threshold = 0.0015):
    df['Signal'] = 0

    for i in range(1, len(df)):
        # Previous candle conditions
        prev_candle_closes_below_bb = df['Close'].iloc[i-1] < df['bb_low'].iloc[i-1]
        prev_rsi_below_thr = df['rsi'].iloc[i-1] < rsi_threshold_low
        # Current candle conditions
        closes_above_prev_high = df['Close'].iloc[i] > df['High'].iloc[i-1]
        bb_width_greater_threshold = df['bb_width'].iloc[i] > bb_width_threshold

        # Combine conditions
        if (prev_candle_closes_below_bb and
            prev_rsi_below_thr and
            closes_above_prev_high and
            bb_width_greater_threshold):
            df.at[i, 'Signal'] = 2  # Set the buy signal for the current candle

        # Previous candle conditions
        prev_candle_closes_above_bb = df['Close'].iloc[i-1] > df['bb_up'].iloc[i-1]
        prev_rsi_above_thr = df['rsi'].iloc[i-1] > rsi_threshold_high
        # Current candle conditions
        closes_below_prev_low = df['Close'].iloc[i] < df['Low'].iloc[i-1]
        bb_width_greater_threshold = df['bb_width'].iloc[i] > bb_width_threshold

        # Combine conditions
        if (prev_candle_closes_above_bb and
            prev_rsi_above_thr and
            closes_below_prev_low and
            bb_width_greater_threshold):
            df.at[i, 'Signal'] = 1  # Set the sell signal for the current candle
    
    return df

In [ ]:
def strategy_02(df, current_candle):
    current_pos = df.index.get_loc(current_candle)
    c0 = df['Open'].iloc[current_pos] > df['Close'].iloc[current_pos]
    # Condition 1: The high is greater than the high of the previous day
    c1 = df['High'].iloc[current_pos] > df['High'].iloc[current_pos - 1]
    # Condition 2: The low is less than the low of the previous day
    c2 = df['Low'].iloc[current_pos] < df['Low'].iloc[current_pos - 1]
    # Condition 3: The close of the Outside Bar is less than the low of the previous day
    c3 = df['Close'].iloc[current_pos] < df['Low'].iloc[current_pos - 1]

    if c0 and c1 and c2 and c3:
        return 2  # Signal for entering a Long trade at the open of the next bar
    
    c0 = df['Open'].iloc[current_pos] < df['Close'].iloc[current_pos]
    # Condition 1: The high is greater than the high of the previous day
    c1 = df['Low'].iloc[current_pos] < df['Low'].iloc[current_pos - 1]
    # Condition 2: The low is less than the low of the previous day
    c2 = df['High'].iloc[current_pos] > df['High'].iloc[current_pos - 1]
    # Condition 3: The close of the Outside Bar is less than the low of the previous day
    c3 = df['Close'].iloc[current_pos] > df['High'].iloc[current_pos - 1]
    
    if c0 and c1 and c2 and c3:
        return 1

    return 0

def add_signal_2(df):
    df['Signal'] = df.progress_apply(lambda row: strategy_02(df, row.name), axis=1)
    return df

In [ ]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    if all(relevant_rows['High'] < relevant_rows['EMA']):
        return 1
    elif all(relevant_rows['Low'] > relevant_rows['EMA']):
        return 2
    else:
        return 0

def add_ema_signal(df):
    df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 5) if row.name >= 20 else 0, axis=1)
    return df

In [ ]:
def add_signal_by_ema_macd(df):
    def add_signal(df, current_candle, backcandles):
        if (ema_signal(df, current_candle, backcandles)==2 and
            all(df.loc[current_candle - 3:current_candle - 2, "MACD"] < df.loc[current_candle - 3:current_candle - 2, "MACD_signal"]) and
            all(df.loc[current_candle - 1:current_candle, "MACD"] > df.loc[current_candle - 1:current_candle, "MACD_signal"]) ):
            return 2
        if (ema_signal(df, current_candle, backcandles)==1 and
            all(df.loc[current_candle - 3:current_candle - 2, "MACD"] > df.loc[current_candle - 3:current_candle - 2, "MACD_signal"]) and
            all(df.loc[current_candle - 1:current_candle, "MACD"] < df.loc[current_candle - 1:current_candle, "MACD_signal"]) ):

            return 1
        return 0
        
    df['Signal'] = df.progress_apply(lambda row: add_signal(df, row.name, 7) if row.name != 0 else 0, axis=1)
    return df